In [14]:
import psutil
import numpy as np
import pandas as pd
import re 
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx
import time
import plotly.io as pio

print('start', time.ctime())
network = nx.Graph()

df = pd.read_csv('filtered_jan.csv', sep=",")
total_row = df.shape[0]

# load edges and nodes
for i in range(total_row):
    subset = df.iloc[i]['hashtags']
    tokens = str(subset).split(' ')
    for j in range(len(tokens)):
        for k in range(j+1, len(tokens)):
            if network.has_edge(tokens[j], tokens[k]):
                network[tokens[j]][tokens[k]]['weight'] += 1
            else:
                network.add_edge(tokens[j], tokens[k])
                network[tokens[j]][tokens[k]]['weight'] = 1

nx.set_node_attributes(network, 2, 'size')

                
pos_ = nx.spring_layout(network)
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')
edge_trace = []
for edge in network.edges():
    if 'weight' in network.edges()[edge]:
        tag_1 = edge[0]
        tag_2 = edge[1]
        x0, y0 = pos_[tag_1]
        x1, y1 = pos_[tag_2]
        text   = tag_1 + '--' + tag_2 + ': ' + str(network.edges()[edge]['weight'])
        trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                           width = 0.3*network.edges()[edge]['weight']**1.75)
    edge_trace.append(trace)
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in network.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*network.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])
    
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
# Add node trace
fig.add_trace(node_trace)
# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)


pio.orca.config.executable = '/Users/yannading/miniconda3/bin/orca'
py.plot(fig, filename='filtered_jan.html')
fig.show()

print('end', time.ctime())

start Wed Jul 22 02:00:01 2020


end Wed Jul 22 02:00:05 2020
